# What makes Julia special

Julia's key attraction is that it's a dynamic, interactive language that runs as fast as C or Fortran. How does this work? How does Julia achieve these seemingly contradictory goals when other languages have not? The answer is 

  * **A strict type system**
  * **Type inference** (Julia figures out the types of variables)
  * **Multiple dispatch** (Julia makes multiple versions of the same function for different argument types)
  * **Just-in-time compilation** (Julia compiles the different functions to machine code at run time)

Other languages like Java and some versions of Python have just-in-time (JIT) compilation, which in some cases can result in dynamic user code running as fast as compiled code. But JIT is an afterthought to those languages, and it often can't be applied. Julia was designed from the ground up to make JIT work whenever possible. 

## Just-in-time compilation

Just-in-time compilation is the run-time transformation of high-level code (the kind of code people write) to machine code (the low-level instructions that computer chips understand). To see how this works in Julia, let's define a simple function $f(x) = 4x(1-x)$ (the logistic map) and transform it in stages to machine code. 

In [1]:
function f(x) 
    4x*(1-x)
end

f (generic function with 1 method)

How fast does this function run? Find out with the `@time` macro

In [2]:
@time f(0.3)
@time f(0.3)

  0.002432 seconds (484 allocations: 29.433 KiB)
  0.000001 seconds (5 allocations: 176 bytes)


0.84

Whoa! The first run is three thousand times slower than the second! Why? It's because Julia compiles `f` to machine code during its first execution. This happens in four steps which we can examine using Julia's *code introspection macros*.

### 1. Transformation of user code to  abstract syntax tree

**@code_lowered** shows the *abstract syntax tree* (AST) for `f(x) = 4x(1-x)`. This is a computer-friendly representation of same high-level function but with all the implicit steps spelled out explicitly, e.g. the implicit multiplication in `4x` gets turned into the more explicit `4*x`.

In [3]:
@code_lowered(f(1.0))

CodeInfo(:(begin 
        nothing
        return (4 * x) * (1 - x)
    end))

### 2. Specification of types

**`@code_typed`** further breaks down the abstract syntax tree by specifying *exactly which operations to execute for for a particular argument type*, here `Float64`. This is an importasnt step because computer chips execute floating-point and integer arithmetic with different hardware. 

In [4]:
@code_typed(f(1.0))

CodeInfo(:(begin 
        return (Base.mul_float)((Base.mul_float)((Base.sitofp)(Float64, 4)::Float64, x)::Float64, (Base.sub_float)((Base.sitofp)(Float64, 1)::Float64, x)::Float64)::Float64
    end))=>Float64

The output of @code_typed is not easy to read, but you can see two multiplications (Base.mul_float) and one subtraction (Base.sub_float), which makes sense for f(x) = 4x(1-x).

### 3. Transformation to intermediate compiler language

**@code_llvm** further transforms the AST into an `intermediate compiler language` for the LLVM compiler. Intermediate compiler languages (also called `assembly languages`) are a sort of generic representation of the very lowest-level machine code, just one step of away from the hardware instructions that run on particular chips.

In [5]:
@code_llvm(f(1.0))


define double @julia_f_61004(double) #0 !dbg !5 {
top:
  %1 = fmul double %0, 4.000000e+00
  %2 = fsub double 1.000000e+00, %0
  %3 = fmul double %1, %2
  ret double %3
}


With a little scrutiny you can see below that calculation of `f(x) = 4x(1-x)` above in four steps: 
  * multiply `%0` (the function argument `x`) by 4, then save in `%1` (`%1 = 4x`)
  * subtract `%0` from 1, save in `%2` (`%2 = 1-x`)
  * multiply `%1` and `%2`, save in `%3` (`%3x = 4x(1-x)`)
  * return `%3`
 
Note that the arithmetic operations are specified for the particular numeric type: `fmul double` means "floating-point multiplication, double precision". This corresponds to a particular hardware instruction.

### 4.  Transformation to machine code

**@code_native** performs the final transformation of the intermediate compiler language to *machine code*, that is a set of instructions for you particular chipset that are executed directly in hardware.

In [6]:
@code_native(f(1.0))

	.text
Filename: In[1]
	pushq	%rbp
	movq	%rsp, %rbp
	movabsq	$140717366775464, %rax  # imm = 0x7FFB50A90EA8
	movabsq	$140717366775472, %rcx  # imm = 0x7FFB50A90EB0
Source line: 2
	movsd	(%rcx), %xmm1           # xmm1 = mem[0],zero
	subsd	%xmm0, %xmm1
	mulsd	(%rax), %xmm0
	mulsd	%xmm1, %xmm0
	popq	%rbp
	retq
	nopw	(%rax,%rax)
